In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [ ]:
!pip install selenium

In [103]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

# 4.Crawling

스크레이핑(Scraping)이라고도 하며 웹 페이지 내의 데이터를 추출하는 것을 의미 <br>
데이터를 수집하기 위한 방법으로 많이 사용 <br>
크게 두 가지의 방법이 존재
1. 정적크롤링 <br>
정적 데이터를 수집하는 방법 <br>
정적 데이터란 페이지 내에 원하는 정보가 모두 들어남

2. 동적크롤링 <br>
동적 데이터를 수집하는 방법 <br>
동적 데이터란 클릭, 로그인 등의 행위를 통해 원하는 데이터에 접근 가능 <br>


|    |정적 크롤링|동적 크롤링|
|----|--------|--------|
|방법 |주소 사용  |브라우저 사용|
|수집 범위|제한적  |제한 없음|
|속도|매우 빠름|매우 
<br>

크롤링 시 사이트에서 크롤링을 허용하는지를 반드시 확인해야 함 <br>
robots.txt를 뒤에 붙여 확인 가능 <br>
강제는 아니나 이를 무시하면 추후 법률적 문제가 생길 수 있음
```
www.daum.net/robots.txt
User-agent: *
Disallow: /
```
*: All <br>
/: All Directories


## 4.1 HTTP

WWW(World Wide Web, W3) 상에서 정보를 주고받을 수 있는 프로토콜 <br>
클라이언트와 서버 사이에 이루어지는 요청/응답 프로토콜

### API
API(Application Programming Interface) <br>
- Application: 고유한 기능을 가진 모든 소프트웨어
- Interface: 두 애플리케이션 간의 규약 <br>
이 계약은 요청과 응답을 사용하여 두 애플리케이션이 서로 통신하는 방법을 정의합니다.

### REST
REST(Representational State Transfer): 자원을 이름으로 구분하여 해당 자원의 상태를 주고받는 것 <br>

REST 구성
1. 자원(Resource): HTTP URI
2. 자원에 대한 행위(Verb): HTTP Method
3. 자원에 대한 행위의 내용(Representations): HTTP Message Pay Load


- HTTP URI(Uniform Resource Identifier)를 통해 자원(Resource)을 명시
- HTTP Method(POST, GET, PUT, DELETE)를 사용하여 URI에 대한 CRUD Operation을 적용 <br>
<br>

HTTP Methods
- GET: 자원 검색
- POST: 자원 작성
- PUT: 자원 업데이트
- DELETE: 데이터 삭제
- HEAD: 자원 검색 (GET과 유사하나 상태 줄과 헤더만 반환)
- OPTIONS: 자원이 지원하고 있는 메소드의 취득
- PATCH: 자원 일부 수정 (PUT과 유사하나 일부만 수정)
- CONNECT: 자원의 터널 접속을 변경
- TRACE: 리소스에 대한 경로를 따라 메시지 루프백 테스트를 수행
<br>

HTTP Status
1. 1xx(Informational): 요청 처리중
2. 2xx(Successful): 요청 정상 처리 <br>
200: 요청 성공
3. 3xx(Redirection): 요청을 완료하려면 추가 행동이 필요
4. 4xx(Client Error): 클라이언트 오류, 잘못된 문법등으로 요청을 수행할 수 없음 <br>
400: Bad Request, 클라이언트의 잘못된 요청으로 서버가 요청을 처리할 수 없음 <br>
401: Unauthorized, 해당 리소스에 대한 인증이 필요함 <br>
403: Forbidden, 서버가 요청을 이해했지만 승인을 거부함 <br>
404: Not Found, 리소스를 찾을 수 없음 <br>
5. 5xx(Server Error): 서버 오류

### REST API
REST의 원리를 따르는 API <br>
※ RESTful: REST의 원리를 따르는 시스템

## 4.2 HTML
HTML(HyperText Markup Language)은 웹 페이지 표시를 위해 개발된 지배적인 마크업 언어 <br>
HTML은 웹 페이지 콘텐츠 안의 꺾쇠 괄호에 둘러싸인 "태그"로 되어있는 HTML 요소 형태로 작성 <br>
HTML은 웹 브라우저와 같은 HTML 처리 장치의 행동에 영향을 주는 자바스크립트, 본문과 그 밖의 항목의 외관과 배치를 정의하는 CSS 같은 스크립트를 포함하거나 불러올 수 있음 <br>
<br>
HTML 선택자: HTML에서는 다수의 동일한 태그가 존재하는데 각 태그를 구별할 수 있도록 선택자를 이용
```html
<div> 
	<div> 
      <a> c </a> 
      <span> c++ </span> 
    </div> 
    
    <div> 
      <a> java </a> 
      <span> python </span> 
    </div> 
</div>
```

```html
<div id="contents"> 
	<div class="data1"> 
      <span class="language"> c++ </span> 
      <span class="language"> java </span> 
      <span class="language"> python </span> 
  </div> 
    
  <div class="data2"> 
      <a class="framework"> tensorflow </a> 
      <a class="framework"> pytorch </a> 
      <a class="framework"> spring </a> 
  </div> 
</div>
```

## 4.3 정적크롤링

#### 4.3.1 라이브러리

##### 4.3.1.1 requests

requests: 파이썬용 http 라이브러리 <br>
reference: https://requests.readthedocs.io/en/latest/

메소드별 사용법
```python
GET: requests.get()
POST: requests.post()
PUT: requests.put()
DELETE: requests.delete()
```

```python
import requests

requests.get("https://jsonplaceholder.typicode.com/users/1")
```

In [ ]:
import requests
from bs4 import BeautifulSoup

response = requests.get("https://jsonplaceholder.typicode.com/users/1")
bs = BeautifulSoup(response.text, 'lxml')
bs

###### Response Body

요청이 정상적으로 처리가 되면, response body에 요청한 데이터가 담겨져 옴. <br>
response body 크게 3가지 방식으로 읽을 수 있음 <br>
1. content: binary 원문을 읽음
```python
response.content
```
2. text: utf-8로 인코딩 된 문자열로 읽음
```python
response.text
```
3. json: 응답이 json이면 dict로 읽음
```python
response.json()
```



###### Request

param: 주소에 포함된 변수를 담음<br>
ex) https://www.naver.com/post/12345 <br>
-> 12345 <br>
query: 주소 바깥? 이후의 주소를 담음<br>
ex) https://www.naver.com/post/post_id=12345&id=1 <br>
-> 12345, 1
body: XML, JSON 등의 데이터를 담음, 주소에서는 확인 불가<br>
<br>

requests에서는 아래와 같이 사용
- get
```python
response = requests.get("https://naver.com/post", params={"post_id": "12345", "id": "1"})
```

- post, put: HTML 데이터 전송
```python
response = requests.get("https://naver.com/post", data={"post_id": "12345", "id": "1"})
```
json 형태로도 요청 가능
```python
response = requests.get("https://naver.com/post", json={"post_id": "12345", "id": "1"})
```

###### headers

일부 웹 사이트는 bot agent를 차단 <br>
이 경우 header의 user-agent를 아래와 같이 넘기면 해결 <br>
```python
requests.get("https://naver.com/post", headers={'User-Agent': 'Mozilla 5.0'})
```

##### 4.3.1.2 BeautifulSoup

BeautifulSoup: html, xml 등으로부터 원하는 정보를 가지고 올 수 있도록 하는 라이브러리 <br>
reference: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

```python
import requests
from bs4 import BeautifulSoup

response = requests.get(url)
bs = BeautifulSoup(response.text, 'lxml')
```

###### Parser

|parser|특징|설치|속도|사용방법|
|------|---|---|---|------|
|html.parser||기본|보통|BeautifulSoup(html_doc, 'html.parser')|
|lxml|xml 지원|lxml 필요|빠름|BeautifulSoup(html_doc, 'lxml')|
|xml|xml 지원|lxml 필요|빠름|BeautifulSoup(html_doc, 'xml')|
|html5lib|브라우저와 동일|html5lib 필요|느림|BeautifulSoup(html_doc, 'html5lib')|

###### find
속성과 값을 이용하여 원하는 값을 찾음

find: 매칭되는 값 중 상위 1개 반환
find_all: 매칭되는 전체 반환

특정 태그 추출
```python
soup.find_all('p') # p 태그 추출
```
<br>

특정 클래스 추출
```python
soup.find_all(class_='a') # a 클래스 추출
```
<br>

특정 태그와 class 추출
```python
soup.find_all('p', attrs={'class': 'a'}) # p 태그와 a 클래스 모두를 갖는 값 추출
```
<br>

특정 id 추출
```python
soup.find_all(id='b') # b id를 갖는 값 추출
```
<br>

###### select
CSS Selector로 태그를 찾아 반환 <br>
CSS에서 HTML을 태깅하는 방법을 활용 <br>
<br>
select_one: 매칭되는 값 중 상위 1개 반환 <br>
select: 매칭되는 전체 반환 <br>
<br>

특정 태그 추출
```python
soup.select('p') # p 태그 추출
```
<br>

특정 클래스 추출
```python
soup.select('.a') # a 클래스 추출
```
<br>

특정 태그와 class 추출
```python
soup.select('p.a') # p 태그와 a 클래스 모두를 갖는 값 추출
```
<br>

특정 id 추출
```python
soup.select('#b') # b id를 갖는 값 추출
```
<br>

특정 태그와 id 추출
```python
soup.select('p#b') # p 태그와 b id 모두를 갖는 값 추출
```
<br>

특정 태그와 class, id 모두 추출
```python
soup.select('p.a#b') # p 태그와 a 클래스 b id 모두 갖는 값 추출
```
<br>

특정 태그 아래에 있는 태그 찾기
```python
soup.select('div p') # div 아래 p태그가 있는 값 추출
soup.select('div > p') # div 바로 아래 p태그가 있는 값 추출
soup.select("div > #link") # div 바로 아래 link id가 있는 값 추출
```
<br>

형제 태그 찾기
```python
soup.select("#link + .sister") # link 태그와 형제 태그 중 바로 직후 1개
soup.select("#link ~ .sister") # link 태그와 형제 태그 중 뒤에 태그 전부
```
<br>

여러 태그 중 i번째 태그 추출
```python
soup.select('a:nth-of-type(i)') # 추출된 a태그 중 i번째 값 반환
```
<br>
<br>

정규표현식 활용 <br>
```python
soup.select('[class~=a]') # class 속성 중 a를 포함하는 태그
soup.select('a[href]') # a 태그 중 href 속성이 존재하는 태그
soup.select('a[href="https://www.naver.com"]') # a 태그 중 href 속성이 https://www.naver.com과 매칭되는 태그
soup.select('a[href^="https://"]') # a 태그 중 href 속성이 https://로 시작하는 태그
soup.select('a[href$="ac.kr"]') # a 태그 중 href 속성이 ac.kr로 끝나는 태그
soup.select('a[href*="naver"]') # a 태그 중 href 속성 중 naver를 가지는 태그
```

<br>
<br>

출력
```python
soup.strings # 값 반환
soup.stripped_strings # 공백을 제거한 값 반환
```

### 4.3.2 실습

##### 예제

```html
<div id="contents"> 
    <div class="data1"> 
      <span class="language"> c++ </span> 
      <span class="language"> java </span> 
      <span class="language"> python </span> 
  </div> 

  <div class="data2"> 
      <a class="framework"> tensorflow </a> 
      <a class="framework"> pytorch </a> 
      <a class="framework"> spring </a> 
  </div> 
</div>
```

In [ ]:
response = '''
<div id="contents"> 
  <div class="data1"> 
      <span class="language"> c++ </span> 
      <span class="language"> java </span> 
      <span class="language"> python </span> 
  </div> 

  <div class="data2"> 
      <a class="framework"> tensorflow </a> 
      <a class="framework"> pytorch </a> 
      <a class="framework"> spring </a> 
  </div> 
</div>
'''

In [ ]:
bs = BeautifulSoup(response, 'lxml')

In [ ]:
bs.select('div')[0]

In [ ]:
bs.select('div')[1]

In [ ]:
bs.select('div')[2]

In [ ]:
bs.select('div')[2].select('a')

In [ ]:
bs.select('div')[2].select('a')[0]

In [ ]:
bs.select('div')[2].select('a')[1]

In [ ]:
bs.select('div')[2].select('a')[2]

In [ ]:
bs.select('div > span')

In [ ]:
bs.select('div > a')

In [ ]:
bs.select('div > a')[0].text.strip()

In [ ]:
bs.select('div > a')[0].text.strip()

In [ ]:
[tag.text.strip() for tag in bs.select('div > a')]

In [ ]:
bs.select('#contents')

In [ ]:
bs.select('div#contents')

In [ ]:
bs.select('div.data2')

#### 네이버 뉴스

개별 뉴스

In [ ]:
response = requests.get(
    'https://n.news.naver.com/mnews/article/023/0003759424',
    headers={
    'User-Agent': 'Mozilla 5.0'
    })
bs = BeautifulSoup(response.text, 'lxml')

In [ ]:
title = bs.select('h2#title_area > span')[0].text.strip()
time = bs.select('span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME')[0].text.strip()
author = bs.select('em.media_end_head_journalist_name')[0].text.strip()
contents = bs.select('#newsct_article')[0].text.strip()

한 페이지 내용 수집

In [ ]:
def get_bs_from_url(url):
    response = requests.get(
        url,
        headers={
        'User-Agent': 'Mozilla 5.0'
        }
    )

    bs = BeautifulSoup(response.text, 'lxml')

    return bs

In [ ]:
bs = get_bs_from_url('https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=023')

In [ ]:
titles = [tag.text.strip() for tag in bs.select('div.list_body.newsflash_body li dt')[1::2]]
urls = [tag.select('a')[0].get('href') for tag in bs.select('div.list_body.newsflash_body li dt')[1::2]]

In [ ]:
bs.select('div.list_body.newsflash_body li')

In [ ]:
news = pd.DataFrame({
    'title': titles,
    'url': urls
})

In [ ]:
news

페이지 전체

In [ ]:
total_news = pd.DataFrame(columns=['title', 'url'])
for page in range(1, 8):
    url = f'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=023&date=20230422&page={page}'
    bs = get_bs_from_url(url)

    titles = [tag.text.strip() for tag in bs.select('div.list_body.newsflash_body li dt')[1::2]]
    urls = [tag.select('a')[0].get('href') for tag in bs.select('div.list_body.newsflash_body li dt')[1::2]]

    news = pd.DataFrame({
    'title': titles,
    'url': urls
    })

    total_news = pd.concat([total_news, news])

특정 날짜 범위

In [ ]:
total_news = pd.DataFrame(columns=['date', 'title', 'url'])
for date in tqdm(pd.date_range('20230401', '20230423')):
    date = date.strftime("%Y%m%d")

    url = f'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=023&date={date}&page=10000'
    bs = get_bs_from_url(url)
    last_page_num = int(bs.select('.paging > strong')[0].text.strip())

    for page in range(1, last_page_num+1):
        url = f'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=023&date={date}&page={page}'
        bs = get_bs_from_url(url)

        titles = [tag.text.strip() for tag in bs.select('div.list_body.newsflash_body li dt')[1::2]]
        urls = [tag.select('a')[0].get('href') for tag in bs.select('div.list_body.newsflash_body li dt')[1::2]]

        news = pd.DataFrame({
        'date': date,
        'title': titles,
        'url': urls
        })

        total_news = pd.concat([total_news, news])

In [ ]:
total_news

여러 언론사

In [ ]:
total_news = pd.DataFrame(columns=['date', 'oid', 'title', 'url'])
for oid in tqdm(['023', '028']):
    for date in pd.date_range('20230401', '20230423'):
        date = date.strftime("%Y%m%d")

        url = f'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid={oid}&date={date}&page=10000'
        bs = get_bs_from_url(url)
        last_page_num = int(bs.select('.paging > strong')[0].text.strip())

        for page in range(1, last_page_num+1):
            url = f'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid={oid}&date={date}&page={page}'
            bs = get_bs_from_url(url)

            titles = [tag.text.strip() for tag in bs.select('div.list_body.newsflash_body li dt')[1::2]]
            urls = [tag.select('a')[0].get('href') for tag in bs.select('div.list_body.newsflash_body li dt')[1::2]]

            news = pd.DataFrame({
            'date': date,
            'oid': oid,
            'title': titles,
            'url': urls
            })

            total_news = pd.concat([total_news, news])

In [ ]:
total_news

Naver News Crawler

In [ ]:
def naver_news_crawler(oids, start_date, end_date):
    total_news = pd.DataFrame(columns=['date', 'oid', 'title', 'url'])

    for oid in tqdm(oids):
        for date in pd.date_range(start_date, end_date):
            date = date.strftime("%Y%m%d")

            url = f'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid={oid}&date={date}&page=10000'
            bs = get_bs_from_url(url)
            last_page_num = int(bs.select('.paging > strong')[0].text.strip())

            for page in range(1, last_page_num+1):
                url = f'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid={oid}&date={date}&page={page}'
                bs = get_bs_from_url(url)

                titles = [tag.text.strip() for tag in bs.select('div.list_body.newsflash_body li dt')[1::2]]
                urls = [tag.select('a')[0].get('href') for tag in bs.select('div.list_body.newsflash_body li dt')[1::2]]

                news = pd.DataFrame({
                'date': date,
                'oid': oid,
                'title': titles,
                'url': urls
                })

                total_news = pd.concat([total_news, news])

    return total_news

In [ ]:
naver_news_crawler(['023', '025', '028'], '20230421', '20230423')

#### 네이버 증권

In [5]:
response = requests.get('https://finance.naver.com/item/main.naver?code=086520#')
bs = BeautifulSoup(response.text, 'lxml')

In [25]:
current_price = int(bs.select('div.today > p.no_today')[0].select('span.blind')[0].text.strip().replace(',', ''))

In [24]:
int(current_price.replace(',', ''))


730000

#### 다음 증권

In [35]:
import json

In [27]:
response = requests.get('https://finance.daum.net/domestic/market_cap')
bs = BeautifulSoup(response.text, 'lxml')

In [28]:
bs.select('div.leftW')

[<div class="leftW">
 <div class="titB">
 <h3>시가총액상위</h3>
 <span id="boxMarketStatus"></span>
 </div>
 <table class="tab">
 <colgroup>
 <col width="100"/>
 <col width="100"/>
 <col width="*"/>
 </colgroup>
 <tbody>
 <tr>
 <td class="on"><a href="?market=KOSPI" title="코스피">코스피</a></td>
 <td class=""><a href="?market=KOSDAQ" title="코스닥">코스닥</a></td>
 <td class="last">
 <a class="btnChart" href="/domestic/chart_slides/market_cap?market=KOSPI" title="차트슬라이드"><i>-</i>차트슬라이드</a>
 </td>
 </tr>
 </tbody>
 </table>
 <div class="tableB mt" id="boxMarketCap"></div>
 <div class="AdvertB borderT" id="boxSponsor"></div>
 </div>]

In [33]:
headers = {
    'User-Agent': 'Mozilla 5.0',
    'referer': 'https://finance.daum.net/domestic/market_cap',
}
response = requests.get(
    'https://finance.daum.net/api/trend/market_capitalization?page=1&perPage=30&fieldName=marketCap&order=desc&market=KOSPI&pagination=true',
    headers=headers
)
bs = BeautifulSoup(response.text, 'lxml')

In [34]:
bs

<html><body><p>{"data":[{"rank":1,"name":"삼성전자","symbolCode":"A005930","code":"KR7005930003","tradePrice":65500.0,"change":"RISE","changePrice":900.0,"changeRate":0.0139318885,"marketCap":391020757025000.0,"listedShareCount":5969782550,"foreignRatio":"0.518"},{"rank":2,"name":"LG에너지솔루션","symbolCode":"A373220","code":"KR7373220003","tradePrice":581000.0,"change":"FALL","changePrice":6000.0,"changeRate":-0.0102214651,"marketCap":135954000000000.0,"listedShareCount":234000000,"foreignRatio":"0.054"},{"rank":3,"name":"SK하이닉스","symbolCode":"A000660","code":"KR7000660001","tradePrice":89500.0,"change":"RISE","changePrice":700.0,"changeRate":0.0078828829,"marketCap":65156211667500.0,"listedShareCount":728002365,"foreignRatio":"0.495"},{"rank":4,"name":"삼성바이오로직스","symbolCode":"A207940","code":"KR7207940008","tradePrice":781000.0,"change":"RISE","changePrice":3000.0,"changeRate":0.0038560411,"marketCap":55586894000000.0,"listedShareCount":71174000,"foreignRatio":"0.106"},{"rank":5,"name":"LG화학"

In [36]:
headers = {
    'User-Agent': 'Mozilla 5.0',
    'referer': 'https://finance.daum.net/domestic/market_cap',
}
response = requests.get(
    'https://finance.daum.net/api/trend/market_capitalization?page=1&perPage=30&fieldName=marketCap&order=desc&market=KOSPI&pagination=true',
    headers=headers
)
bs = json.loads(response.text)

In [38]:
print(bs.keys())

dict_keys(['data', 'totalCount', 'totalPages', 'currentPage', 'pageSize', 'baseDate'])


In [43]:
pd.DataFrame(bs.get('data'))

,rank,name,symbolCode,code,tradePrice,change,changePrice,changeRate,marketCap,listedShareCount,foreignRatio
0,1,삼성전자,A005930,KR7005930003,65500.0,RISE,900.0,0.013932,3.910208e+14,5969782550,0.518
1,2,LG에너지솔루션,A373220,KR7373220003,581000.0,FALL,6000.0,-0.010221,1.359540e+14,234000000,0.054
2,3,SK하이닉스,A000660,KR7000660001,89500.0,RISE,700.0,0.007883,6.515621e+13,728002365,0.495
3,4,삼성바이오로직스,A207940,KR7207940008,781000.0,RISE,3000.0,0.003856,5.558689e+13,71174000,0.106
4,5,LG화학,A051910,KR7051910008,740000.0,FALL,1000.0,-0.001350,5.223833e+13,70592343,0.484
5,6,삼성SDI,A006400,KR7006400006,691000.0,FALL,15000.0,-0.021246,4.751629e+13,68764530,0.494
6,7,삼성전자우,A005935,KR7005931001,55700.0,RISE,600.0,0.010889,4.583479e+13,822886700,0.720
7,8,현대차,A005380,KR7005380001,197500.0,FALL,4500.0,-0.022277,4.177747e+13,211531506,0.316
8,9,기아,A000270,KR7000270009,84500.0,FALL,2100.0,-0.024249,3.425320e+13,405363347,0.370
9,10,POSCO홀딩스,A005490,KR7005490008,377000.0,FALL,10000.0,-0.025840,3.188335e+13,84571230,0.408


In [51]:
headers = {
    'User-Agent': 'Mozilla 5.0',
    'referer': 'https://finance.daum.net/domestic/market_cap',
}
response = requests.get(
    'https://finance.daum.net/api/trend/market_capitalization?page=1&perPage=100&fieldName=marketCap&order=desc&market=KOSDAQ&pagination=true',
    headers=headers
)
bs = json.loads(response.text)
pd.DataFrame(bs.get('data'))

,rank,name,symbolCode,code,tradePrice,change,changePrice,changeRate,marketCap,listedShareCount,foreignRatio
0,1,에코프로비엠,A247540,KR7247540008,267000.0,FALL,2500.0,-0.009276,2.611296e+13,97801344,0.097
1,2,에코프로,A086520,KR7086520004,730000.0,RISE,21000.0,0.029619,1.943820e+13,26627668,0.065
2,3,셀트리온헬스케어,A091990,KR7091990002,69400.0,FALL,600.0,-0.008571,1.141331e+13,164456910,0.162
3,4,엘앤에프,A066970,KR7066970005,265000.0,FALL,11000.0,-0.039855,9.544854e+12,36018316,0.231
4,5,HLB,A028300,KR7028300002,35350.0,RISE,350.0,0.010000,4.333089e+12,122576784,0.149
...,...,...,...,...,...,...,...,...,...,...,...
95,96,유진테크,A084370,KR7084370006,28850.0,FALL,50.0,-0.001730,6.611278e+11,22916042,0.218
96,97,콜마비앤에이치,A200130,KR7200130003,22300.0,EVEN,0.0,0.000000,6.588213e+11,29543558,0.024
97,98,네이처셀,A007390,KR7007390008,10500.0,FALL,400.0,-0.036697,6.579001e+11,62657156,0.044
98,99,보로노이,A310210,KR7310210000,52000.0,FALL,500.0,-0.009524,6.576892e+11,12647869,0.033


코스피 전체 데이터프레임 만들기

In [62]:
headers = {
    'User-Agent': 'Mozilla 5.0',
    'referer': 'https://finance.daum.net/domestic/market_cap',
}
response = requests.get(
    'https://finance.daum.net/api/trend/market_capitalization?page=1&perPage=30&fieldName=marketCap&order=desc&market=KOSPI&pagination=true',
    headers=headers
)
bs = json.loads(response.text)

total_stock = pd.DataFrame(bs.get('data'))

In [64]:
for page in tqdm(range(2, bs.get('totalPages')+1)):
    response = requests.get(
        f'https://finance.daum.net/api/trend/market_capitalization?page={page}&perPage=30&fieldName=marketCap&order=desc&market=KOSPI&pagination=true',
        headers=headers
    )

    bs = json.loads(response.text)
    temp_stock = pd.DataFrame(bs.get('data'))

    total_stock = pd.concat([total_stock, temp_stock])

100%|██████████| 66/66 [00:13<00:00,  4.90it/s]


In [65]:
total_stock

,rank,name,symbolCode,code,tradePrice,change,changePrice,changeRate,marketCap,listedShareCount,foreignRatio
0,1,삼성전자,A005930,KR7005930003,65500.0,RISE,900.0,0.013932,3.910208e+14,5969782550,0.518
1,2,LG에너지솔루션,A373220,KR7373220003,581000.0,FALL,6000.0,-0.010221,1.359540e+14,234000000,0.054
2,3,SK하이닉스,A000660,KR7000660001,89500.0,RISE,700.0,0.007883,6.515621e+13,728002365,0.495
3,4,삼성바이오로직스,A207940,KR7207940008,781000.0,RISE,3000.0,0.003856,5.558689e+13,71174000,0.106
4,5,LG화학,A051910,KR7051910008,740000.0,FALL,1000.0,-0.001350,5.223833e+13,70592343,0.484
...,...,...,...,...,...,...,...,...,...,...,...
17,1998,QV 블룸버그 2X 천연가스 선물 ETN(H),Q550074,KRG550000746,1305.0,RISE,25.0,0.019531,1.957500e+09,1500000,0.000
18,1999,ARIRANG TDF2050액티브,A433870,KR7433870003,10535.0,RISE,95.0,0.009100,1.896300e+09,180000,0.000
19,2000,KBSTAR 200생활소비재,A287330,KR7287330005,6300.0,RISE,45.0,0.007194,1.386000e+09,220000,0.000
20,2001,KBSTAR 200커뮤니케이션서비스,A315480,KR7315480004,11505.0,RISE,220.0,0.019495,1.380600e+09,120000,0.000


## 4.4 동적크롤링

reference: https://selenium-python.readthedocs.io/index.html

### 4.4.1 설치 방법

```cmd
pip install selenium
```

Chrome Driver: https://chromedriver.chromium.org/downloads

### 4.4.2 사용법

기본적인 사용 방법은 아래와 같음
```python
from selenium import webdriver
 
driver = webdriver.Chrome(executable_path='chromedriver') # executable_path: chromedriver path
# driver = webdriver.Firefox()

driver.get(url="http://www.naver.com")

driver.close()
```

#### 4.4.2.1 Locating Elements

find_element: 조건에 해당하는 하나의 값 반환  
find_elements: 조건에 해당하는 다수의 값 반환

element를 찾는 method는 아래와 같음
- by_ID
- by_xpath
- by_link_text
- by_partial_link_text
- by_name
- by_tag_name
- by_class_name
- by_css_selector

```python
element = driver.find_element(By.XPATH, x_path)
element = driver.find_element_by_xpath(x_path)

element = driver.find_element(By.CSS_SELECTOR, css)
element = driver.find_element_by_css_selector(css)
```

해당 값을 못 찾은 경우 NoSuchElementException 발생

##### Send Keys
```python
driver.find_element_by_xpath(xpath).send_keys(id)
driver.find_element_by_xpath(xpath).send_keys(password)
driver.find_element_by_xpath(xpath).click()
```

외에도 Keys 내의 다양한 커맨드 입력 가능
```
from selenium.webdriver.common.keys import Keys
```
|Command          |Action |
|-----------------|-------|
|Keys.ENTER       |엔터    |
|Keys.RRETURN     |       |
|Keys.SPACE       |스페이스 |
|Keys.ARROW_UP    |화살표   |
|Keys.ARROW_DOWN  |       |
|Keys.ARROW_LEFT  |       |
|Keys.ARROW_RIGHT |       |
|Keys.BACK_SPACE  |지우기   |
|Keys.DELETE      |       |
|Keys.CONTROL     |Ctrl   |
|Keys.ALT         |Alt    |
|Keys.SHIFT       |Shift  |
|Keys.TAB         |Tab    |
|Keys.PAGE_UP     |Page-Up|
|Keys.PAGE_DOWN   |Page-Down|
|Keys.TAB         |Tab    |
|Keys.F1 ~ Keys.F9|F1~F9  |
|Keys.ESCAPE      |ESC    |
|Keys.HOME        |Home   |
|Keys.INSERT      |Insert |
|...              |...    |

##### Drag and Drop

```python
from selenium.webdriver import ActionChains

element = driver.find_element(By.NAME, "source")
target = driver.find_element(By.NAME, "target")

action_chains = ActionChains(driver)
action_chains.drag_and_drop(element, target).perform()
```

##### Moving between Windows

```python
previous_window = driver.window_handles[0]
new_window = driver.window_handles[1]
driver.switch_to.window(new_window)
```


##### Moving

```python
driver.forward()
driver.back()
```

#### 4.4.2.3 Waiting

Selenium은 실행 중인 driver를 이용하여 정보를 추출  
driver에 데이터 로딩이 완료되지 않을 경우 데이터 수집이 불가  
따라서 페이지 이동 등의 작업이 들어가면 다음 작업이 완료될 때까지 대기해야만 함

웹페이지마다 다를수있음 임의로 정함


##### Time

가장 기본적인 방법으로 대기 시간을 time 모듈을 통해 명시

```python
import time
from selenium import webdriver


driver = webdriver.Chrome(executable_path='chromedriver')
driver.get(url="http://www.naver.com")
time.time(5)
driver.close()
```

In [ ]:
help(time.time)

import time
from selenium import webdriver


driver = webdriver.Chrome(executable_path='chromedriver')
driver.get(url="http://www.naver.com")
time.sleep(2)
driver.close()


time.sleep으로 페이지데이터가 전부로딩될때까지 기다리는 코드가 한세트


In [ ]:
driver = webdriver.Chrome(executable_path='chromedriver')
driver.get(url="https://finance.daum.net/domestic/market_cap")
time.sleep(2)
driver.close()


In [ ]:
driver.page_source
#requests에서 return받는 문자열임(response)


##### Implicit Wait

위의 time에서는 2가지 문제가 존재
1. 모든 반응형 동작마다 time을 걸어 웹 로딩 대기
2. time을 5초로 설정했지만 실제 그보다 동작이 빨리 끝날 경우 불필요한 시간 대기

위의 문제를 해결하고자 implicit wait을 이용  
이는 driver의 옵션 설정으로 반응형 동작에서 최대 허용 대기 시간을 설정  
만약 허용 시간 이내에 로드가 완료될 경우 다음 작업을 바로 진행

```python
from selenium import webdriver


driver = webdriver.Chrome(executable_path='chromedriver')
driver.implicitly_wait(time_to_wait=5)
driver.get(url="http://www.naver.com")
driver.close()

```

##### Explicit Wait

implicit wait에서는 driver가 로딩을 대기하는 시간을 설정하여 5초를 설정하였어도 이전에 로드가 완료되면 다음 작업의 진행이 가능하였음  
하지만 위 경우도 문제가 있는데 5초가 지나도 내가 원하는 데이터가 로드가 안 되었을 경우 데이터를 수집할 수 없음  
따라서 특정 값이 로드될 때까지 기다렸다가 그 값이 로드되면 다음 작업을 진행할 필요성이 발생  
이 때 이용하는 게 explicit wait

```python
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome(executable_path='chromedriver') 
driver.get(url="http://www.naver.com")

try:
    element = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.CLASS_NAME , 'paging'))
    )
finally:
    driver.quit()

driver.close()

```

until: 조건이 False인 동안에 계속 실행  
not_until: 조건이 True인 동안에 계속 실행

In [ ]:
#내pc에서 path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome(executable_path='./utils/chromedriver') 
driver.get(url="http://www.naver.com")
#url에 엔터친상황이랑 같음


try:
    element = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.CLASS_NAME , 'paging'))
    )
finally:
    driver.quit()

driver.close()

#### 4.4.2.4 ActionChains

연속 동작을 수행하기 위함

ex) control + c
```python
ActionChains(driver).key_down(Keys.CONTROL).send_keys('c').key_up(Keys.CONTROL).perform()
```

##### Drag and Drop

```python
from selenium.webdriver import ActionChains

element = driver.find_element(By.NAME, "source")
target = driver.find_element(By.NAME, "target")

action_chains = ActionChains(driver)
action_chains.drag_and_drop(element, target).perform()
```

#### 4.4.2.5 Others

##### Options
```python
options = webdriver.ChromeOptions()
options.add_argument('window-size=1920,1080')

driver = webdriver.Chrome(executable_path, options=options)
```

##### Alert

경고창 발생 시 이에 수락, 거절 등의 행동을 취할 수 있음

```python
from selenium.webdriver.common.alert import Alert

Alert(driver).accept() # 수락
Alert(driver).dismiss() # 거절
Alert(driver).send_keys(keysToSend=key) # 특정 키를 보낼 수 있음
```

##### Scroll Down
페이지의 최하단으로 이동
```python
driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
```
<br>

특정 태그가 등장할 때까지 이동
```python
from selenium.webdriver import ActionChains

some_tag = driver.find_element_by_id('gorio')
ActionChains(driver).move_to_element(some_tag).perform()
```

##### Minimize/Maximize

```python
driver.minimize_window()
driver.maximize_window()
```

##### Screen Shot
```python
driver.save_screenshot('screenshot.png')
```

In [70]:
driver = webdriver.Chrome(executable_path='./utils/chromedriver')

C:\Users\user\AppData\Local\Temp\ipykernel_75772\2928108173.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./utils/chromedriver')


In [86]:
driver.get('https://finance.daum.net/domestic/market_cap')
time.sleep(2)

In [89]:
response = driver.page_source
bs = BeautifulSoup(response, 'lxml')

In [95]:
bs.select('div.box_contents tbody')

[<tbody><tr class="first"><td class="first"><span class="time">1</span></td>
 <td><a class="txt" href="/quotes/A005930">삼성전자</a></td>
 <td>
 <span class="num" data-realtime-trade-price="yes">
         65,500
     </span>
 </td>
 <td>
 <span class="num up" data-realtime-change-price="yes">
 <i>▲</i>900
     </span>
 </td>
 <td>
 <span class="num up" data-realtime-change-ratio="yes">
         +1.39%
     </span>
 </td>
 <td>
 <span class="num">
         3,910,208
     </span>
 </td>
 <td>
 <span class="num">
         5,969,782,550
     </span>
 </td>
 <td class="pR">
 <span class="num">
         0.52%
     </span>
 </td>
 </tr><tr><td class="first"><span class="time">2</span></td>
 <td><a class="txt" href="/quotes/A373220">LG에너지솔루션</a></td>
 <td>
 <span class="num" data-realtime-trade-price="yes">
         581,000
     </span>
 </td>
 <td>
 <span class="num down" data-realtime-change-price="yes">
 <i>▼</i>6,000
     </span>
 </td>
 <td>
 <span class="num down" data-realtime-change-ratio=

네이버 검색

In [96]:
driver.get('https://www.naver.com')
time.sleep(1)

In [100]:
# 입력
(
    driver.find_element(
        By.XPATH,
        '/html/body/div[2]/div[2]/div[1]/div/div[3]/form/fieldset/div/input'
    ).send_keys('빅데이터')
)

# 클릭
(
    driver.find_element(
        By.XPATH,
        '/html/body/div[2]/div[2]/div[1]/div/div[3]/form/fieldset/button/span[2]'
    ).click()
)

In [106]:
# 입력
(
    driver.find_element(
        By.XPATH,
        '/html/body/div[2]/div[2]/div[1]/div/div[3]/form/fieldset/div/input'
    ).send_keys('빅데이터')
)

# 엔터
(
    driver.find_element(
        By.XPATH,
        '/html/body/div[2]/div[2]/div[1]/div/div[3]/form/fieldset/div/input'
    ).send_keys(Keys.ENTER)
)


네이버 로그인

In [110]:
# 클릭
(
    driver.find_element(
        By.XPATH,
        '/html/body/div[2]/div[3]/div[3]/div/div[2]/a'
    ).click()
)

# 입력
(
    driver.find_element(
        By.XPATH,
        '/html/body/div[1]/div[2]/div/div[1]/form/ul/li/div/div[1]/div[1]/input'
    ).send_keys('id')
)

# 입력
(
    driver.find_element(
        By.XPATH,
        '/html/body/div[1]/div[2]/div/div[1]/form/ul/li/div/div[1]/div[2]/input'
    ).send_keys('pwd')
)

# 클릭
(
    driver.find_element(
        By.XPATH,
        '/html/body/div[1]/div[2]/div/div[1]/form/ul/li/div/div[7]/button'
    ).click()
)

기업집단포털

In [123]:
driver.get('https://www.egroup.go.kr/egps/wi/mainPage.do')
time.sleep(3)

# 클릭
(
    driver.find_element(
        By.XPATH,
        '/html/body/div[3]/div/div[1]/div[2]/ul[2]/li[2]/a'
    ).click()
)
time.sleep(1)


# 클릭
(
    driver.find_element(
        By.XPATH,
        '/html/body/div[2]/div[3]/div[3]/div/div[2]/div/ul/li[1]/a'
    ).click()
)
time.sleep(3)

response = driver.page_source
bs = BeautifulSoup(response, 'lxml')

In [124]:
bs.select('#resultListDiv td')

[<td name="gbnTd1" rowspan="2">202205</td>,
 <td name="gbnTd2" rowspan="2">76</td>,
 <td>상호출자제한집단(47)</td>,
 <td style="text-align: left;">삼성, 에스케이, 현대자동차, 엘지, 롯데, 포스코, 한화, 지에스, 현대중공업, 농협, 신세계, 케이티, 씨제이, 한진, 카카오, 두산, 엘에스, DL, 부영, 중흥건설, 미래에셋, 네이버, 에쓰-오일, 현대백화점, 에이치엠엠, 금호아시아나, 하림, 에이치디씨, 효성, 영풍, 셀트리온, 교보생명보험, 호반건설, SM, 넷마블, 케이티앤지, 케이씨씨, 대우조선해양, 넥슨, DB, 태영, 코오롱, 오씨아이, 두나무, 세아, 한국타이어, 이랜드</td>,
 <td>공시대상기업집단(29)</td>,
 <td style="text-align: left;">태광, 금호석유화학, 장금상선, 동원, 한라, 쿠팡, 삼천리, 다우키움, 아모레퍼시픽, 엠디엠, 동국제강, 크래프톤, 삼양, 애경, 대방건설, 중앙, 한국항공우주산업, 한국지엠, 하이트진로, 반도홀딩스, 현대해상화재보험, 유진, 보성, KG, 아이에스지주, 일진, 오케이금융그룹, 신영, 농심</td>,
 <td name="gbnTd1" rowspan="2">202105</td>,
 <td name="gbnTd2" rowspan="2">71</td>,
 <td>상호출자제한집단(40)</td>,
 <td style="text-align: left;">삼성, 현대자동차, 에스케이, 엘지, 롯데, 포스코, 한화, 지에스, 현대중공업, 농협, 신세계, 케이티, 씨제이, 한진, 두산, 엘에스, 부영, 카카오, DL, 미래에셋, 현대백화점, 금호아시아나, 에쓰-오일, 셀트리온, 한국투자금융, 교보생명보험, 네이버, 에이치디씨, 효성, 영풍, 하림, 케이티앤지, 케이씨씨, 넥슨, 대우조선해양, 넷마블, 호반건설, SM, DB, 코오롱</td>,
 <td>공시대상기업집단(31)</td>,